# 0. Install and Import Dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 3. Calculate Angles

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [22]:
cap = cv2.VideoCapture('Cric2.mp4')
   
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
#         # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            cv2.putText(image, 'Body Joint Angle : ', (100,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            
            # Visualize angle
            if angle >= 90 and angle <= 115:
                cv2.putText(image, str(int(angle)),(160, 130),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (127, 255, 0), 3, cv2.LINE_AA)
                
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(50,50,245), thickness=3, circle_radius=3), 
                                mp_drawing.DrawingSpec(color=(127,255,0), thickness=3, circle_radius=3)) 
            elif angle > 115:
                cv2.putText(image, str(int(angle)),(160, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (50, 50, 245), 3, cv2.LINE_AA)
                
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(50,50,245), thickness=3, circle_radius=3), 
                                mp_drawing.DrawingSpec(color=(50,50,245), thickness=3, circle_radius=3))
            else:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(50,50,245), thickness=3, circle_radius=3), 
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=3, circle_radius=3))
                       
        except:
            pass              
        
        cv2.imshow('Video Analytics', image)
        
        if cv2.waitKey(10) == ord('p'):
            cv2.waitKey(-1)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# 4. Curl Counter

In [17]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
import tempfile

In [6]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [7]:
cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

0.0

In [8]:
cap.read()

(False, None)

In [9]:
frame

array([[[218, 221, 219],
        [217, 220, 218],
        [215, 218, 216],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [228, 231, 229]],

       [[218, 221, 219],
        [217, 220, 218],
        [215, 218, 216],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [228, 231, 229]],

       [[218, 221, 219],
        [217, 220, 218],
        [214, 217, 215],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [228, 231, 229]],

       ...,

       [[234, 237, 235],
        [234, 237, 235],
        [234, 237, 235],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [229, 232, 230]],

       [[234, 237, 235],
        [234, 237, 235],
        [234, 237, 235],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [229, 232, 230]],

       [[234, 237, 235],
        [234, 237, 235],
        [234, 237, 235],
        ...,
        [228, 231, 229],
        [228, 231, 229],
        [229, 232, 230]]

In [10]:
cap = cv2.VideoCapture('Cric_.mp4')

In [11]:
cap.read()

(True,
 array([[[255, 254, 216],
         [253, 251, 213],
         [152, 152, 109],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[249, 247, 209],
         [154, 152, 114],
         [106, 106,  63],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[134, 141,  77],
         [ 94, 101,  37],
         [ 93, 101,  34],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[198, 222, 212],
         [198, 222, 212],
         [198, 222, 212],
         ...,
         [ 88, 138, 112],
         [ 88, 138, 112],
         [ 89, 139, 113]],
 
        [[252, 255, 253],
         [252, 255, 253],
         [252, 255, 253],
         ...,
         [ 91, 141, 115],
         [ 91, 141, 115],
         [ 91, 141, 115]],
 
        [[248, 251, 249],
         [248, 251, 249],
         [249, 252, 250],
         ...,
         [ 92, 142, 1

In [12]:
cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

568.0

In [13]:
cap.get(cv2.CAP_PROP_FRAME_WIDTH)

1280.0

In [14]:
cap.get(cv2.CAP_PROP_FPS)

30.0

In [15]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [16]:
height

568